In [1]:
!pip install PyCryptoDome



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.7 MB/s eta 0:00:00


IMPORTING PANDAS TO WORK WITH DATA IN A DATAFRAME

In [2]:
import Crypto
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip '/content/drive/MyDrive/archive (8).zip'

Archive:  /content/drive/MyDrive/archive (8).zip
  inflating: ML.csv                  
  inflating: MLtag.csv               


In [9]:
path = '/content/Banking Dataset Sample.xlsx'

df_ = pd.read_csv('/content/ML.csv') #reading file

# to acess and work with create a dataframe

df = pd.DataFrame(df_)

print(df)

     typeofaction  sourceid  destinationid  amountofmoney  \
0         cash-in     30105          28942         494528   
1         cash-in     30105           8692         494528   
2         cash-in     30105          60094         494528   
3         cash-in     30105          20575         494528   
4         cash-in     30105          45938         494528   
...           ...       ...            ...            ...   
2335     transfer     14945          43793         106907   
2336     transfer      9532          43793         106907   
2337     transfer     27332          43793         106907   
2338     transfer     32685          43793         106907   
2339     transfer     26390          43793         106907   

                     date  isfraud typeoffraud  
0     2019-07-19 14:40:00        1       type1  
1     2019-05-17 14:57:00        1       type1  
2     2019-07-20 13:20:00        1       type1  
3     2019-07-03 14:15:00        1       type1  
4     2019-05-26 10:40

In [10]:
 df = df.drop(['date', 'typeoffraud','typeofaction','isfraud'], axis=1) #dropping unnecessary columns
 print(df)

      sourceid  destinationid  amountofmoney
0        30105          28942         494528
1        30105           8692         494528
2        30105          60094         494528
3        30105          20575         494528
4        30105          45938         494528
...        ...            ...            ...
2335     14945          43793         106907
2336      9532          43793         106907
2337     27332          43793         106907
2338     32685          43793         106907
2339     26390          43793         106907

[2340 rows x 3 columns]


In [11]:
df.describe() #describes about the data - mean 1st and 3rd quartile,etc..

,sourceid,destinationid,amountofmoney
count,2340.000000,2340.000000,2.340000e+03
mean,49111.255128,52257.918803,2.508583e+06
std,29614.440603,28988.847246,2.560434e+06
min,61.000000,53.000000,1.333200e+04
25%,24033.000000,28046.000000,3.359140e+05
50%,46632.500000,52748.500000,1.162354e+06
75%,75107.000000,78030.000000,4.686559e+06
max,100000.000000,99950.000000,7.952497e+06


In [12]:
df.dtypes

sourceid         int64
destinationid    int64
amountofmoney    int64
dtype: object

In [13]:
df['sourceid'] = df['sourceid'].astype(str)
df['destinationid'] = df['destinationid'].astype(str)
df['amountofmoney'] = df['amountofmoney'].astype(str)


In [14]:
df.dtypes

sourceid         object
destinationid    object
amountofmoney    object
dtype: object

In [15]:
df.loc[1]

sourceid          30105
destinationid      8692
amountofmoney    494528
Name: 1, dtype: object

GETTING THE KEY AND PROCEEDING WITH ENCODING AND DECODING

In [16]:
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import PBKDF2 # brute force protection
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad,unpad

In [17]:
simple_key = get_random_bytes(32)
print(simple_key)
'''Note- When the following snippet is executed we get a random set of bits every time which is unique from the previous execution.
We have already obtained the random bytes which are used as a key to encrypt the data.
Re- running the snippet may lead to error in the execution of the following functions. Thus if re-run, we will have to change the key value in the below functios.'''

b"B\x19EqX=\x1e\xde\xe3\x1c\x08\xda5\x7f\x8eP'\xa7\xe6\xfa\xdd\xc6\x82+4\x05\xf6\x04\x009r\x8f"


In [33]:
pwd = 'unknown123'
encrypt_salt = b"B\x19EqX=\x1e\xde\xe3\x1c\x08\xda5\x7f\x8eP'\xa7\xe6\xfa\xdd\xc6\x82+4\x05\xf6\x04\x009r\x8f"


In [21]:
def Encrypt_(message):
  encrypt_salt = b"B\x19EqX=\x1e\xde\xe3\x1c\x08\xda5\x7f\x8eP'\xa7\xe6\xfa\xdd\xc6\x82+4\x05\xf6\x04\x009r\x8f"
  pwd = 'unknown123'
#generating keys for encryption
  keys = PBKDF2(pwd, encrypt_salt, dkLen=32)
  cipher = AES.new(keys, AES.MODE_CBC)
  cipher_data = cipher.encrypt(pad(message, AES.block_size)) #encryption
  with open ('encrypted_.bin', 'ab') as f:
    f.write(cipher.iv)
    f.write(cipher_data)
  return 'data stored'



In [20]:
def Decrypt_(bin_file):
  encrypt_salt = b"B\x19EqX=\x1e\xde\xe3\x1c\x08\xda5\x7f\x8eP'\xa7\xe6\xfa\xdd\xc6\x82+4\x05\xf6\x04\x009r\x8f"
  keys = PBKDF2(pwd , encrypt_salt, dkLen=32)
  with open(bin_file,'rb') as f:
    iv = f.read(16)
    decrypt_data = f.read()
  cipher = AES.new(keys, AES.MODE_CBC, iv=iv)
  original = unpad(cipher.decrypt(decrypt_data),AES.block_size)
  return original


In [22]:
# we are looping for first 100 rows of data from the data frame
# we can specify any number till the end of the data frame
for i in range(0,2340):
  string = str(df.loc[i])
  message = b'xyz'
  x_string = message.decode('utf-8')
  x_string_replaced = x_string.replace('xyz', string )
  x_replaced_bytes = x_string_replaced.encode('utf-8')
  Encrypt_(x_replaced_bytes)




In [23]:
Decrypt_('encrypted_.bin')

b'sourceid          30105\ndestinationid     28942\namountofmoney    494528\nName: 0, dtype: object\x02\x02\n\xc4\xb5o\x7f"Y_\xed\xc1R^r\x03x\x14sourceid          30105\ndestinationid      8692\namountofmoney    494528\nName: 1, dtype: object\x02\x02fBX j#\xc0,$\xc5;!\x82\xd2\xea<sourceid          30105\ndestinationid     60094\namountofmoney    494528\nName: 2, dtype: object\x02\x02y\xd6Z\x88\xb3\xce4\xce\\Q\xbb\x7fZ\xb7\xd8rsourceid          30105\ndestinationid     20575\namountofmoney    494528\nName: 3, dtype: object\x02\x02\x84\xcdq\xb9y\xa3?\x1c\x07xv\x99\t\x020\x85sourceid          30105\ndestinationid     45938\namountofmoney    494528\nName: 4, dtype: object\x02\x02B\x12H\xe3\xc4I\x94\xfb\xc8\xeca\xf4F\xb31\x84sourceid          30105\ndestinationid     54971\namountofmoney    494528\nName: 5, dtype: object\x02\x02p\x16\xc4e_\x14\n\xb14\xce^\xbd\xf8\xbb\x9bTsourceid          30105\ndestinationid     62257\namountofmoney    494528\nName: 6, dtype: object\x02\x02\x06\xa41j\xd7\x

Storing key in the binary file for later access and security, Thus altering functions therefore making it usabe for any data-key pairs.

In [28]:
#storing the key in a binary file
keys = PBKDF2(pwd, encrypt_salt, dkLen=32)
encrypt_salt = b'\x02e&/\xa2|\xd0\xbfx\xe99\xb8L\x10h\xde\x83\xbby\x8aXeL(JWc\x00J\xb3\xa6n'
pwd = 'unknown123'
with open('key.bin','ab') as fil:
  fil.write(keys)


In [29]:
def Encrypt_2(message,bin_file):
  with open(bin_file,'rb') as f:
    key = f.read()
  cipher = AES.new(keys, AES.MODE_CBC)
  cipher_data = cipher.encrypt(pad(message, AES.block_size)) #encryption
  with open ('encrypted_2.bin', 'ab') as f:
    f.write(cipher.iv)
    f.write(cipher_data)
  return 'data stored'


In [30]:
#The above written functions can be altered as follows
#this alows us to use the code for any combination of key and data values
#only if the key is known the data can be decrypted
def Decrypt_2(bin_file,bin_filekey):
  with open(bin_filekey,'rb') as f:
    key = f.read()
  with open(bin_file,'rb') as f:
    iv = f.read(16)
    decrypt_data = f.read()
  cipher = AES.new(key, AES.MODE_CBC, iv=iv)
  original = unpad(cipher.decrypt(decrypt_data),AES.block_size)
  return original

In [31]:
for i in range(0,2340):
  string = str(df.loc[i])
  message = b'xyz'
  x_string = message.decode('utf-8')
  x_string_replaced = x_string.replace('xyz', string )
  x_replaced_bytes = x_string_replaced.encode('utf-8')
  Encrypt_2(x_replaced_bytes,'key.bin')

In [32]:
#Now accessing the key from the binary file instead of having it in the function


Decrypt_2('encrypted_2.bin','key.bin')
#if wrong key is used, wrong data is retrived

b'sourceid          30105\ndestinationid     28942\namountofmoney    494528\nName: 0, dtype: object\x02\x02\x99*\xb6sY,\x82\xae\xba\x10\xb01\xcav\xa6\xbasourceid          30105\ndestinationid      8692\namountofmoney    494528\nName: 1, dtype: object\x02\x02\x1c\xc7\r\xcdwj\xa3\x8e\xda\x8d\xce\xec\x83\xf4\xad\nsourceid          30105\ndestinationid     60094\namountofmoney    494528\nName: 2, dtype: object\x02\x02\x8b\xa2\x18\xa0\x8f\x05PvVdS9\xadr\x96:sourceid          30105\ndestinationid     20575\namountofmoney    494528\nName: 3, dtype: object\x02\x02u4u\xc8B\xfa\xba\xd9\x83qD\x05\x96\x8f\xe0\xecsourceid          30105\ndestinationid     45938\namountofmoney    494528\nName: 4, dtype: object\x02\x02\xbf\x0e\x9fw\xfe\xf4\xfadR\x82\xb4\xec\x9c\xf4\x93psourceid          30105\ndestinationid     54971\namountofmoney    494528\nName: 5, dtype: object\x02\x02Q\x92|\x7f\x8bO\xea\xba\xa4\xfe\xfd\x03\xe7!\xf3?sourceid          30105\ndestinationid     62257\namountofmoney    494528\nName: 